# Contexte du projet
On souhaite obtenir des informations à partir du fichier XML sur les chargeurs TESLA, produit dans le brief XML n°2.

---

# Modalités pédagogiques
On cherche à lire et exploiter le fichier XML sur les chargeurs TESLA (créé dans un brief précédent), à partir d'un programme Python.

# 1. Recherche de solution </br>
A partir du site https://docs.python.org/3/library/xml.etree.elementtree.html , et des chapitres "XML tree and elements" et "Parsing XML" de la section "Tutorial", faire les exemples proposés pour découvrir les différentes instructions permettant la lecture d'un fichier XML.

### Parsing XML

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('country_data.xml')
root = tree.getroot()

print(root.tag, root.attrib)

for child in root:
    print(child.tag, child.attrib)

print(root[0][2].text)

### Finding interesting elements

In [ ]:
for neighbor in root.iter('neighbor'):
    print(neighbor.attrib)

for country in root.findall('country'):
    gdppc = country.find('gdppc').text
    name = country.get('name')
    print(name, gdppc)

### Modifying an XML File

In [ ]:
for rank in root.iter('rank'):
    rank.text = str(int(rank.text) + 1)
    rank.set('updated', 'yes')

tree.write('output.xml')

for country in root.findall('country'):
    rank = int(country.find('rank').text)
    if rank > 50:
        root.remove(country)

tree.write('output.xml')

### Building XML documents

In [ ]:
a = ET.Element('a')
b = ET.SubElement(a, 'b')
c = ET.SubElement(a, 'c')
d = ET.SubElement(c, 'd')
ET.dump(a)

### Parsing XML with Namespaces

In [ ]:
root = ET.fromstring(
    """<?xml version="1.0"?>
    <actors xmlns:fictional="http://characters.example.com"
            xmlns="http://people.example.com">
        <actor>
            <name>John Cleese</name>
            <fictional:character>Lancelot</fictional:character>
            <fictional:character>Archie Leach</fictional:character>
        </actor>
        <actor>
            <name>Eric Idle</name>
            <fictional:character>Sir Robin</fictional:character>
            <fictional:character>Gunther</fictional:character>
            <fictional:character>Commander Clement</fictional:character>
        </actor>
    </actors>"""
)

for actor in root.findall('{http://people.example.com}actor'):
    name = actor.find('{http://people.example.com}name')
    print(name.text)
    for char in actor.findall('{http://characters.example.com}character'):
        print(' |-->', char.text)

---

# 2. Transposer ces instructions à la lecture du fichier XML sur les chargeurs TESLA. Faire quelques essais pour, par exemple, afficher la liste des chargeurs.

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('chargeursTesla.xml')
root = tree.getroot()

for chargeur in root.findall('chargeur'):
    station = chargeur.find('n_station').text
    id = chargeur.get('id')
    print(f"ID : {id} => Station : {station[18:]}")

---

# 3. Ecrire un programme qui demander la saisie d'un département, puis affiche le ou les chargeurs de ce département.

In [ ]:
def recherche_dep(dep):

    trouver = False


    for chargeur in root.findall('chargeur'):
        if chargeur.find('code_insee').text[:2] == dep:
            print(f"Chargeur dans le département N°{dep} : {chargeur.find('n_station').text}.\n")
            trouver = True

    if not trouver:
        print(f"Pas de chargeur dans le département N°{dep}.\n")

def menu():
    while True:
        try:
            entree = int(input("Saisissez un numéro de département:\n"))

        except ValueError:
            print("Entrez un numéro valide !\n")
            continue

        recherche_dep(str(entree))

        entree = input("Voulez-vous continuer ? (yes)\n")
        if entree.find('yes') == -1:
            break
            
menu()

---

# 4. A partir d'une liste contenant le numéro des départements d'une région donnée (la franche-comté par exemple):

## 4.1. lister les chargeurs se trouvant dans cette région

In [ ]:
def recherche_reg(reg):

    for dep in reg:
        trouver = False
        for chargeur in root.findall('chargeur'):
            if chargeur.find('code_insee').text[:2] == dep:
                print(f"Chargeur dans le département N°{dep} : {chargeur.find('n_station').text}.\n")
                trouver = True
        if not trouver:
            print(f"Pas de chargeur dans le département N°{dep}.\n")

def menu():
    while True:
        try:
            nbr_dep = int(input("Saisissez le nombre de départements dans votre région:\n"))
            entree = [str(int(input("Saisissez un de numéro de département:\n"))) for _ in range(nbr_dep)]

        except ValueError:
            print("Entrez un numéro valide !\n")
            continue

        recherche_reg(entree)

        entree = input("Voulez-vous continuer ? (yes)\n")
        if entree.find('yes') == -1:
            break
            
menu()

## 4.2 calculer le nombre total de prises disponibles ainsi que la puissance cumulée

In [ ]:
def recherche_dep(reg):
    
    somme_prise = 0
    somme_puissance = 0

    for chargeur in root.findall('chargeur'):

        if chargeur.find('code_insee').text[:2] in reg:
            somme_prise += int(chargeur.find('nbre_pdc').text)
            somme_puissance += int(chargeur.find('puiss_max').text) * int(chargeur.find('nbre_pdc').text)
    
    if somme_prise != 0:
        print(f"Nombre de prises dans cette région: {somme_prise} pour une puissance cumulée de {somme_puissance} Watts.\n")

    else:
        print("Pas de chargeurs dans ces départements\n")
    
def menu():
    while True:

        try:
            nbr_dep = int(input("Combien y a-t-il de départements dans votre région ?:\n"))
            entree = [str(int(input("Saisissez un des numéros de département de votre région:\n"))) for _ in range(nbr_dep)]

        except ValueError:
            print("Entrez un numéro valide !\n")
            continue

        recherche_dep(entree)

        entree = input("Voulez-vous continuer ? (yes)\n")
        if entree.find('yes') == -1:
            break
            
menu()

---

# 5. Demander la saisie d'un numéro d'autoroute, puis afficher, s'il y en a, la liste des chargeurs pour cet autoroute ainsi que ses caractéristiques. 
S'il y a plusieurs chargeurs, la liste sera présentée du nord au sud (on utilisera les coordonnées GPS pour ce classement).

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('chargeursTesla2.xml')
root = tree.getroot()

def recherche_chargeur(autoroute):
    liste_lat = []
    for child in root:

        for chargeur in child.findall('chargeur'):
            
            for detail in chargeur.findall('ad_station'):
                
                if f"A{autoroute}" == detail.find('autoroute').text :
                    liste_lat.append(chargeur.find('Ylatitude').text)
        
    liste_lat.sort()
    
    while liste_lat:
        for child in root:

            for chargeur in child.findall('chargeur'):

                if chargeur.find('Ylatitude').text == liste_lat[0]:
                    puissance = 150 if chargeur.find('puiss_max').text == """"1""""" else 250
                    type_prise = "CCS Combo 2 - T2 (spécifique Tesla)" if chargeur.find('type_prise').text == """"1""""" else "CCS Combo 2"
                    print(f"Chargeur sur la A{autoroute}:\nNom: {chargeur.find('n_station').text}\nID: {chargeur.find('id_station').text}\nPuissance: {puissance} Watts\nType de prise: {type_prise}\nLatitude: {chargeur.find('Ylatitude').text}\nLongitude: {chargeur.find('Xlongitude').text}\n\n")

        liste_lat.pop(0)

def menu():
    while True:

        try:
            entree = str(int(input("Saisissez un numéro d'autoroute':\n")))

        except ValueError:
            print("Entrez un numéro valide !\n")
            continue

        recherche_chargeur(entree)

        entree = input("Voulez-vous continuer ? (yes)\n")
        if entree.find('yes') == -1:
            break
            
menu()